In [3]:
import pandas as pd
import py2neo
from dotenv import load_dotenv
import re
import os
from web3 import Web3 
import time
load_dotenv()
from py2neo import Graph

# alchemy_api_key = os.getenv('ALCHEMY_API')
web3 = Web3(Web3.HTTPProvider('https://eth-mainnet.alchemyapi.io/v2/P667s84hD-xZzok8oF7Rhq38ztt7EMCE'));()


()

In [4]:
g = Graph("neo4j+s://98f2d230.databases.neo4j.io", auth=("neo4j", "5q3k_oxiFBncGPNK7gwmd7TfcUym9OmFQRxtZHOpP60"))
def query_neo(query, graph=g):
    response = graph.run(query)
    print(response)
    return(pd.DataFrame(response))

In [5]:

ens_twitter_df = pd.read_csv('data/cleaned_ens_twitter_handles.csv')

In [6]:
ens_twitter_df.head(2)

,Unnamed: 0.1,Unnamed: 0,ens_type,ens,twitter_id,valid_ens,owner_wallet,id,handle
0,0,0,name_ens,sourav.eth,3200004419,True,0x7fF204b7d3e825376E5fA067799F8A5aAfE285e5,3200004419,000sourav
1,1,2,name_ens,rafa.eth,2523228612,True,0x4406571181978028dF5621E973CE52cd5F2d05F8,2523228612,01RafaelJimenez


In [10]:
def create_wallet(row):
    wallet = row['owner_wallet']
    wallet_query = f"""
        merge 
            (w:Wallet {{address: '{wallet}'}})
        on create set // so we can delete everything we created
            w:EnsTest
    """
    response = query_neo(wallet_query)
    return(None)

def create_ens(row):
    name = row['ens']
    tokenStandard = 'ERC-721'
    ens_query = f"""
    merge 
        (e:Ens {{name: '{name}'}})
    on create set 
        e:Token 
    on create set 
        e:Nft
    on create set
        e:EnsTest
    on create set  
        e.tokenStandard = '{tokenStandard}'
    on create set
        e.name = '{name}'
    """
    response = query_neo(ens_query)
    return(None)


def create_twitter(row):
    username = row['handle']
    twitter_id = row['twitter_id']
    twitter_query = f"""
    merge 
        (t:Twitter {{twitter_id: '{twitter_id}'}})
    on create set 
        t:Account
    on create set
        t:Nft
    on create set 
        t:EnsTest
    on create set 
        t.username = '{username}'"""
    response = query_neo(twitter_query)
    return(None)
    
def connect_dots(row):
    create_wallet(row)
    create_ens(row)
    create_twitter(row)
    address = row['owner_wallet']
    twitter_id = row['twitter_id']
    ens = row['ens']
    connect_query = f"""
    match 
        (w:Wallet {{address: '{address}'}})
    match 
        (e:Ens {{name: '{ens}'}})
    match 
        (t:Twitter {{twitter_id: '{twitter_id}'}})
    with 
        w, e, t
    merge 
        (w)-[r:OWNER]->(e)
    set 
        r.type = 'registrar'
    set 
        r.source = 'Web3.py'
    merge
        (t)-[r1:HAS_ALIAS]->(e)
    set
        r1.source = 'https://raw.githubusercontent.com/ultrasoundmoney/ens_twitter_accounts/main/ens_twitter.json'
    """
    response = query_neo(connect_query)
    return(None)
    
    

In [8]:
len(ens_twitter_df)

14021

In [11]:
import time
def ingest_queries(cleaned_ens_df):
    counter = 0 
    for index,row in cleaned_ens_df.iterrows():
        counter += 1
        str_counter = str(counter)
        print(f"Doing number {counter} and yes i know i shouldn't be lazy and doing this instead of LOAD CSV")
        connect_dots(row)
    print("i'm done")

ingest_queries(ens_twitter_df)


Doing number 1 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 2 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 3 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 4 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 5 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 6 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 7 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
Doing number 8 and yes i know i shouldn't be lazy and doing this instead of LOAD CSV
(No data)
(No data)
(No data)
(No data)
